In [ ]:
import pandas as pd


df = pd.read_csv("/content/dataset (1).csv")


df["timestamp"] = df["LastUpdatedDate"].astype(str) + " " + df["LastUpdatedTime"].astype(str)


traffic_map = {"low": 2, "average": 5, "high": 8}
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].map(traffic_map).fillna(df["TrafficConditionNearby"]).astype(float)


df = df.rename(columns={
    "SystemCodeNumber": "parking_lot_id",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_length",
    "VehicleType": "vehicle_type",
    "TrafficConditionNearby": "traffic_level",
    "IsSpecialDay": "is_special_day"
})


df_clean = df[[
    "timestamp", "parking_lot_id", "latitude", "longitude", "capacity",
    "occupancy", "queue_length", "vehicle_type", "traffic_level", "is_special_day"
]]


df_clean.to_csv("clean_dataset_final.csv", index=False)
print(" Data cleaned and saved as clean_dataset.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pathway
import pathway as pw
class ParkingRecord(pw.Schema):
    timestamp: str
    parking_lot_id: str
    latitude: float
    longitude: float
    capacity: int
    occupancy: int
    queue_length: int
    vehicle_type: str
    traffic_level: float
    is_special_day: int


input_table = pw.io.csv.read(
    "/content/clean_dataset_final.csv",
    schema=ParkingRecord,
    mode="streaming",
    autocommit_duration_ms=500
)


BASE_PRICE = 10.0
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 2.0}

@pw.udf
def model1_price(occupancy, capacity):
    alpha = 2.0
    return BASE_PRICE + alpha * (occupancy / capacity)

@pw.udf
def model2_price(capacity, occupancy, queue_length, traffic_level, is_special_day, vehicle_type):
    alpha = 3.0
    beta = 1.5
    gamma = 2.0
    delta = 5.0
    vehicle_weight = vehicle_weights.get(vehicle_type.lower(), 1.0)
    demand = (alpha * (occupancy / capacity) + beta * queue_length -
              gamma * traffic_level + delta * is_special_day + vehicle_weight)
    norm_demand = min(max(demand / 100, 0), 1)
    return max(min(BASE_PRICE * (1 + 1.5 * norm_demand), BASE_PRICE * 2), BASE_PRICE * 0.5)

@pw.udf
def model3_price_simple(occupancy, capacity, queue_length, traffic_level, base_price):
    if occupancy >= capacity and queue_length > 2:
        return max(base_price - 1.0, BASE_PRICE * 0.5)
    elif traffic_level > 5:
        return min(base_price + 1.0, BASE_PRICE * 2)
    return base_price


with_prices = input_table.with_columns(
    price_model_1=model1_price(input_table.occupancy, input_table.capacity),
    price_model_2=model2_price(
        input_table.capacity,
        input_table.occupancy,
        input_table.queue_length,
        input_table.traffic_level,
        input_table.is_special_day,
        input_table.vehicle_type
    )
)

model3 = with_prices.with_columns(
    price_model_3=model3_price_simple(
        with_prices.occupancy,
        with_prices.capacity,
        with_prices.queue_length,
        with_prices.traffic_level,
        with_prices.price_model_2
    )
)


pw.io.csv.write(model3, "output.csv")
pw.run()

In [ ]:
df=pd.read_csv('/content/output.csv')
df

In [ ]:
df["revenue_model_1"] = df["price_model_1"] * df["occupancy"]
df["revenue_model_2"] = df["price_model_2"] * df["occupancy"]
df["revenue_model_3"] = df["price_model_3"] * df["occupancy"]
avg_revenue = {
    "Model 1": df["revenue_model_1"].mean(),
    "Model 2": df["revenue_model_2"].mean(),
    "Model 3": df["revenue_model_3"].mean()
}

for model, revenue in avg_revenue.items():
    print(f"{model} Average Revenue: ₹{revenue:.2f}")



In [ ]:
revenue_by_lot = df.groupby("parking_lot_id")[[
    "revenue_model_1", "revenue_model_2", "revenue_model_3"
]].sum().round(2)

revenue_by_lot.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.bar(avg_revenue.keys(), avg_revenue.values(), color=['skyblue', 'lightgreen', 'salmon'])
plt.title("Average Revenue by Pricing Model")
plt.ylabel("Revenue (₹)")
plt.show()

In [ ]:

import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()


df = pd.read_csv("/content/output.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])

lot_id = "BHMBCCMKT01"
df_lot = df[df["parking_lot_id"] == lot_id].sort_values("timestamp")


source = ColumnDataSource(df_lot)


p = figure(x_axis_type="datetime", height=400, width=800,
           title=f"Dynamic Pricing Over Time for Parking Lot {lot_id}",
           tools="pan,wheel_zoom,box_zoom,reset,save")


p.line("timestamp", "price_model_1", source=source, line_width=2, color="blue", legend_label="Model 1")
p.line("timestamp", "price_model_2", source=source, line_width=2, color="green", legend_label="Model 2")
p.line("timestamp", "price_model_3", source=source, line_width=2, color="red", legend_label="Model 3")


p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price (₹)"
p.title.text_font_size = "16pt"


show(p)